In [2]:
import numpy as np
import pymc3 as pm
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd

# Question 1

In [3]:
df=pd.read_csv("data1.csv")
#no need to normalize data
x=df['x']
y=df['y']

# Question 2

In [64]:
df=pd.read_csv("data2_sensor1.csv")
#no need to normalize data
x1=df['temperature']
y1=df['pressure']
df=pd.read_csv("data2_sensor2.csv")
#no need to normalize data
x2=df['temperature']
y2=df['pressure']